# 🧹 Data Preprocessing & Feature Engineering
### Health Insurance Customer Dataset

This notebook cleans, transforms, and prepares the dataset for:
- Customer segmentation (unsupervised ML)
- Fraud detection (supervised ML, added later)

It includes handling missing values, encoding categorical variables, scaling numerical features, and extracting date features.

## 📦 Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

sns.set(style="whitegrid")
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

## 📁 Load Dataset

In [2]:
df = pd.read_csv("../data/insurance_synthetic.csv")
df.head()

,Customer ID,Age,Gender,Marital Status,Occupation,Income Level,Education Level,Geographic Information,Location,Behavioral Data,...,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Risk Profile,Previous Claims History,Credit Score,Driving Record,Life Events,Segmentation Group
0,84966,23,Female,Married,Entrepreneur,70541,Associate Degree,Mizoram,37534,policy5,...,Email,In-Person Meeting,Afternoon,English,1,3,728,DUI,Job Change,Segment5
1,95568,26,Male,Widowed,Manager,54168,Doctorate,Goa,63304,policy5,...,Mail,In-Person Meeting,Morning,French,1,2,792,Clean,Retirement,Segment5
2,10544,29,Female,Single,Entrepreneur,73899,Associate Degree,Rajasthan,53174,policy5,...,Email,Mail,Evening,German,2,1,719,Accident,Childbirth,Segment3
3,77033,20,Male,Divorced,Entrepreneur,63381,Bachelor's Degree,Sikkim,22803,policy5,...,Text,In-Person Meeting,Anytime,French,3,0,639,DUI,Job Change,Segment3
4,88160,25,Female,Separated,Manager,38794,Bachelor's Degree,West Bengal,92858,policy1,...,Email,Text,Weekends,English,0,3,720,Major Violations,Childbirth,Segment2


## 📅 Convert Date Columns & Extract Features

In [3]:
date_cols = ["Policy Start Date", "Policy Renewal Date"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Extract useful date features
df["Policy_Duration_Days"] = (df["Policy Renewal Date"] - df["Policy Start Date"]).dt.days
df["Policy_Start_Year"] = df["Policy Start Date"].dt.year
df["Policy_Start_Month"] = df["Policy Start Date"].dt.month

df.drop(columns=date_cols, inplace=True)

df.head()

,Customer ID,Age,Gender,Marital Status,Occupation,Income Level,Education Level,Geographic Information,Location,Behavioral Data,...,Preferred Language,Risk Profile,Previous Claims History,Credit Score,Driving Record,Life Events,Segmentation Group,Policy_Duration_Days,Policy_Start_Year,Policy_Start_Month
0,84966,23,Female,Married,Entrepreneur,70541,Associate Degree,Mizoram,37534,policy5,...,English,1,3,728,DUI,Job Change,Segment5,124.0,2023.0,8.0
1,95568,26,Male,Widowed,Manager,54168,Doctorate,Goa,63304,policy5,...,French,1,2,792,Clean,Retirement,Segment5,1006.0,2020.0,9.0
2,10544,29,Female,Single,Entrepreneur,73899,Associate Degree,Rajasthan,53174,policy5,...,German,2,1,719,Accident,Childbirth,Segment3,427.0,2023.0,9.0
3,77033,20,Male,Divorced,Entrepreneur,63381,Bachelor's Degree,Sikkim,22803,policy5,...,French,3,0,639,DUI,Job Change,Segment3,NaN,NaN,NaN
4,88160,25,Female,Separated,Manager,38794,Bachelor's Degree,West Bengal,92858,policy1,...,English,0,3,720,Major Violations,Childbirth,Segment2,282.0,2022.0,12.0


## 🔍 Identify Numerical & Categorical Columns

In [4]:
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numerical Columns:\n", numerical_cols)
print("Categorical Columns:\n", categorical_cols)

Numerical Columns:
 ['Customer ID', 'Age', 'Income Level', 'Location', 'Claim History', 'Coverage Amount', 'Premium Amount', 'Deductible', 'Risk Profile', 'Previous Claims History', 'Credit Score', 'Policy_Duration_Days', 'Policy_Start_Year', 'Policy_Start_Month']
Categorical Columns:
 ['Gender', 'Marital Status', 'Occupation', 'Education Level', 'Geographic Information', 'Behavioral Data', 'Purchase History', 'Interactions with Customer Service', 'Insurance Products Owned', 'Policy Type', 'Customer Preferences', 'Preferred Communication Channel', 'Preferred Contact Time', 'Preferred Language', 'Driving Record', 'Life Events', 'Segmentation Group']


## 🧼 Handle Missing Values

In [5]:
null_cols = df.columns[df.isnull().any()]
print(null_cols)

# Numerical: fill with median
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Categorical: fill with mode
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

df[null_cols].isnull().sum()

Index(['Policy_Duration_Days', 'Policy_Start_Year', 'Policy_Start_Month'], dtype='object')


Policy_Duration_Days    0
Policy_Start_Year       0
Policy_Start_Month      0
dtype: int64

## 🔠 Encode Categorical Variables
Using OneHotEncoding for multi-category fields.

In [6]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df_encoded.head()

,Customer ID,Age,Income Level,Location,Claim History,Coverage Amount,Premium Amount,Deductible,Risk Profile,Previous Claims History,...,Driving Record_Major Violations,Driving Record_Minor Violations,Life Events_Divorce,Life Events_Job Change,Life Events_Marriage,Life Events_Retirement,Segmentation Group_Segment2,Segmentation Group_Segment3,Segmentation Group_Segment4,Segmentation Group_Segment5
0,84966,23,70541,37534,5,366603,2749,1604,1,3,...,False,False,False,True,False,False,False,False,False,True
1,95568,26,54168,63304,0,780236,1966,1445,1,2,...,False,False,False,False,False,True,False,False,False,True
2,10544,29,73899,53174,4,773926,4413,1612,2,1,...,False,False,False,False,False,False,False,True,False,False
3,77033,20,63381,22803,5,787815,4342,1817,3,0,...,False,False,False,True,False,False,False,True,False,False
4,88160,25,38794,92858,3,366506,1276,133,0,3,...,True,False,False,False,False,False,True,False,False,False


## 📏 Scale Numerical Features
Standardize numerical columns for clustering and ML models.

In [7]:
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

df_encoded.head()

,Customer ID,Age,Income Level,Location,Claim History,Coverage Amount,Premium Amount,Deductible,Risk Profile,Previous Claims History,...,Driving Record_Major Violations,Driving Record_Minor Violations,Life Events_Divorce,Life Events_Job Change,Life Events_Marriage,Life Events_Retirement,Segmentation Group_Segment2,Segmentation Group_Segment3,Segmentation Group_Segment4,Segmentation Group_Segment5
0,1.161055,-1.401980,-0.333617,-0.636458,1.416283,-0.469361,-0.213640,0.871878,-0.525889,1.212606,...,False,False,False,True,False,False,False,False,False,True
1,1.537483,-1.203033,-0.780348,0.377921,-1.441872,1.071729,-0.822588,0.587999,-0.525889,0.301218,...,False,False,False,False,False,True,False,False,False,True
2,-1.481328,-1.004085,-0.241996,-0.020824,0.844652,1.048219,1.080474,0.886162,0.354840,-0.610170,...,False,False,False,False,False,False,False,True,False,False
3,0.879390,-1.600928,-0.528975,-1.216311,1.416283,1.099966,1.025256,1.252169,1.235569,-1.521558,...,False,False,False,True,False,False,False,True,False,False
4,1.274459,-1.269349,-1.199821,1.541248,0.273021,-0.469722,-1.359210,-1.754449,-1.406619,1.212606,...,True,False,False,False,False,False,True,False,False,False


## 💾 Save Processed Dataset
This dataset will be used for:
- Customer segmentation
- Fraud detection (after adding Fraud_Flag in the next notebook)

In [8]:
df_encoded.to_csv("../data/processed_insurance_dataset.csv", index=False)
print("Processed dataset saved successfully!")

Processed dataset saved successfully!


## ✅ Next Steps

Now that preprocessing is complete, the next notebooks will be:

### 1️⃣ Fraud Detection (Supervised ML)
- Add synthetic fraud label
- Train Random Forest, SVM, KNN
- Evaluate using confusion matrix, ROC curve, AUC

### 2️⃣ Customer Segmentation (Unsupervised ML)
- K-Means clustering
- DBSCAN
- Hierarchical clustering
- PCA visualization
- Cluster profiling

Proceed to the next notebook when ready.